# **Modelling and Evaluating**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

### Directories

In [ ]:
base_dir = "workspace/P5-mildew-detection"
test_dir = "inputs/cherry-leaves/test"
train_dir = "inputs/cherry-leaves/train"
validation_dir = "inputs/cherry-leaves/validation"
cherry_leaves_dir = 'inputs/cherry-leaves'


### Set Output

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Set Labels

In [ ]:
labels = os.listdir(train_dir)
print(labels)

### Import Image Data Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Image Augmentation

In [ ]:
augment = ImageDataGenerator(rotation_range=20,
                            width_shift_range=0.10,
                            height_shift_range=0.10,
                            shear_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest',
                            rescale=1./255
                            )

### Image Size

In [ ]:
image_shape = (100,100,3)

### Augment Train, Validation and Test Image Sets

In [ ]:
batch_size = 50
train_set = augment.flow_from_directory(train_dir,
                                        target_size=image_shape[:2],
                                        color_mode='rgb',
                                        batch_size=batch_size,
                                        class_mode='binary',
                                        shuffle=True
                                        )

validation_set = augment.flow_from_directory(validation_dir,
                                        target_size=image_shape[:2],
                                        color_mode='rgb',
                                        batch_size=batch_size,
                                        class_mode='binary',
                                        shuffle=False
                                        )

test_set = augment.flow_from_directory(test_dir,
                                        target_size=image_shape[:2],
                                        color_mode='rgb',
                                        batch_size=batch_size,
                                        class_mode='binary',
                                        shuffle=False
                                        )

In [ ]:
import matplotlib.pyplot as plt

### Plot Augmentated Images

In [ ]:
for _ in range(3):
    img, label = train_set.next()
    print(image_shape)
    plt.imshow(img[0])
    plt.show()

In [ ]:
for _ in range(3):
    img, label = validation_set.next()
    print(image_shape)
    plt.imshow(img[0])
    plt.show()

In [ ]:
for _ in range(3):
    img, label = train_set.next()
    print(image_shape)
    plt.imshow(img[0])
    plt.show()

### Create CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

def create_model():
    image_shape = (100,100,3)
    model = Sequential()

    model.add(Conv2D(filters=16, kernel_size=(3,3),
                        input_shape=image_shape, activation='relu',))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=32, kernel_size=(3,3),
                        input_shape=image_shape, activation='relu',))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),
                        input_shape=image_shape, activation='relu',))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
create_model().summary()

### Early Stoppage

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=5)

### Prepare Model For Training

In [ ]:
model = create_model()
model.fit(train_set,
        epochs=15,
        validation_data=validation_set,
        verbose=1,
        callbacks=[early_stop]
        )

### Save The Model

In [ ]:
model.save('outputs/v1/Mildew_Detection_Model.h5')

### Load Model

In [ ]:
from keras.models import load_model
model = load_model('outputs/v1/Mildew_Detection_Model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---